In [16]:
# !pip install polars
import polars as pl
import os
pl.Config.set_fmt_str_lengths(400)

polars.config.Config

In [17]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [18]:
fids_df = pl.read_parquet(f"{DATA_PATH}/raw/farcaster-fids-0-1730134800.parquet")

In [19]:
# profiles_df = pl.read_parquet(f"{DATA_PATH}/farcaster-fids-0-1730134800.parquet")farcaster-fids-0-1727974800

In [20]:
links_df = pl.read_parquet(f"{DATA_PATH}/farcaster-links-0-1730134800.parquet")

In [6]:
reactions_lazy_df = pl.scan_parquet(f"{DATA_PATH}/farcaster-reactions-0-1730134800.parquet")

In [7]:
verifications_df = pl.read_parquet(f"{DATA_PATH}/farcaster-verifications-0-1730134800.parquet")

In [8]:
# Verifications count per fid
verifications_count_df = (
    verifications_df
    .group_by("fid")
    .agg(pl.count("id").alias("verifications_count"))
)

In [10]:
# Likes count and recasts count per fid
reactions_count_lazy = (
    reactions_lazy_df
    .filter(pl.col("deleted_at").is_null())  # Filter out deleted reactions
    .group_by(["fid", "reaction_type"])
    .agg(pl.count("id").alias("count"))
)

In [11]:
reactions_count_df = reactions_count_lazy.collect()

In [12]:
reactions_pivot_df = (
    reactions_count_df
    .pivot(
        values="count",
        index="fid",
        columns="reaction_type",
        aggregate_function="first"
    )
    .rename({"1": "likes_count", "2": "recasts_count"})
    .fill_null(0)
)

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_71011/671256064.py:2: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  reactions_count_df


In [13]:
following_count_df = (
    links_df
    .filter((pl.col("type") == "follow") & (pl.col("deleted_at").is_null()))
    .group_by("fid")
    .agg(pl.count("id").alias("following_count"))
)

In [14]:
# Followers count (number of users following the fid)
followers_count_df = (
    links_df
    .filter((pl.col("type") == "follow") & (pl.col("deleted_at").is_null()))
    .group_by("target_fid")
    .agg(pl.count("id").alias("followers_count"))
    .rename({"target_fid": "fid"})
)

In [19]:
result_df = fids_df.join(verifications_count_df, on="fid", how="left")
result_df = result_df.join(following_count_df, on="fid", how="left")
result_df = result_df.join(followers_count_df, on="fid", how="left")
result_df = result_df.join(reactions_pivot_df, on="fid", how="left")
result_df = result_df.fill_null(0)

In [20]:
result_df.write_parquet(f"{DATA_PATH}/aggregated_links_statistics.parquet")

In [27]:
import psutil
import os

# Function to print the current memory usage
def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory = process.memory_info().rss / (1024 * 1024)  # Convert to MB
    print(f"Memory Usage: {memory:.2f} MB")

In [28]:
print_memory_usage()

Memory Usage: 3279.47 MB


In [29]:
result_df.sample(10)

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,recasts_count,likes_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32
2024-07-30 07:46:52.475130,2024-07-30 07:46:52.475130,"b""\xb2?\xc4n\xf4\xd2\xcdxD!.x\x9f\xd9{\x7f\x18nk\xfb""",2024-07-30 07:46:45 UTC,812469,3,91,59,78,559
2024-07-06 09:47:14.356025,2024-07-06 09:47:14.356025,"b"")\xa9\x87+\xf2\x00\x1f\xf6:@\xc6@\xbb\xf1{v\x92\x8c1g""",2024-07-06 09:47:01 UTC,767764,0,72,18,0,0
2024-07-16 14:52:49.646673,2024-07-16 14:52:49.646673,"b""\x1d\x1d=\x0f\x8a\xf5[\xe7\xf0\xbf\x9e\x04\x9f\xc5\xec\xd7N2&\x12""",2024-07-16 14:52:39 UTC,793492,0,108,21,0,8
2023-10-13 16:53:44.963240,2024-06-07 18:11:38.717,"b""^\x18\x88\xa0\xdc\xb9\xb7Y\xbfc\xfe*\xde\x1f\xe1\xfb\x18\x97\x8a\xa7""",2023-11-07 21:27:25 UTC,131238,0,0,0,0,0
2023-10-13 16:18:38.589402,2024-06-07 18:08:33.229,"b""\x07i\x95\xe1:\x048R\xa5\xbf\x1fBWR$\x92\xcb\xbe\xcbn""",2023-11-07 21:08:25 UTC,100504,0,0,0,0,0
2023-10-24 14:07:09.405777,2024-06-07 18:19:27.397,"b""`\x9fk(\xf7,?}\xb0Z\x82\xaa\xe6H\x05h\x0f\xa9\xa7\xa5""",2023-11-07 20:27:51 UTC,191277,4,89,189,22,25
2024-03-21 09:06:25.338336,2024-06-08 03:49:48.246,"b""\x172\x14\xcb:\xcbi\xeb'\xd4I\x04T\xb9\x07\xd9\xd2\x13\x07\xb5""",2024-03-21 09:06:19 UTC,399941,1,76,6,3,2
2023-10-13 15:44:04.898840,2024-06-07 18:05:50.338,"b""\xcf-\xd3Dh\x98P2\xe5\xcb\xfbF\x9b\x1c\xc5\x85\xa2\x19\xb5\x1e""",2023-11-07 20:54:09 UTC,73268,0,0,0,0,0
2023-10-13 15:35:57.055724,2024-06-07 18:05:15.725,"b""\xed\xf0\xa5\xbb\x9ee\x82\xed\xa7\x06\xb9\xf1j,\xa9\xaf\xad\xfe\xe3\x81""",2023-11-07 20:51:13 UTC,67408,0,0,0,0,0


In [21]:
joao_fid = 12031

In [22]:
user_statistics = result_df.filter(pl.col("fid") == joao_fid)


In [23]:
user_statistics

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,recasts_count,likes_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32
2023-09-05 23:18:56.166954,2024-06-07 17:17:31.984,"b""\xb5\x80\x0cCv\x1csPE*\xfa\xc6\xff\x01x\xc1e\xd3\xf2\x06""",2023-11-07 20:14:03 UTC,12031,3,206,4071,36,610


In [34]:
links_df.filter(pl.col("fid") == joao_fid)

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64
860200,1725,"b""\x842\x81\x06\x9b\xefi\xf0]\xb5u\x8f\xc2""\x8e\x17\x95:\x8f\x17""",2024-09-25 15:50:20,2024-09-25 15:50:21.146692,2024-09-25 15:50:21.146692,null,"""follow""",null,11876550315
860200,5650,"b""""\xf4\xfb=\x9c,\xfa\xd2U*?\xa4\x92\x8a\x90*\xb1{\xac\xe9""",2024-09-30 12:55:55,2024-09-30 12:55:55.750938,2024-09-30 12:55:55.750938,null,"""follow""",null,11877929616
860200,191,"b""\xd0PtV\xe4\xd5\xb95\x09@\xc4\x00K\xfe\xd7]\x8dh\x92\xb6""",2024-09-30 19:12:28,2024-09-30 19:12:28.923014,2024-09-30 19:12:28.923014,null,"""follow""",null,11878003533
860200,12031,"b""\xc1\xff\xcbT\x8f\xa4\x9eoi\xe8\xe5y\x19\xa2I\x01mC\xa6T""",2024-10-02 00:43:53,2024-10-02 00:43:53.965456,2024-10-02 00:43:53.965456,null,"""follow""",null,11878298655


In [35]:
links_df.filter(pl.col("target_fid") == joao_fid)

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64
769628,860200,"b""\xf9\xbf\xd9\xa5\xc3Ph\x81\xf8\xb0:u\x1d\xe8@\xb8\xc25\x07b""",2024-09-25 15:45:02,2024-09-25 15:45:15.307277,2024-09-26 15:50:01.807,2024-09-26 15:50:00,"""follow""",null,11876549573
861980,860200,"b""\x10\x11\x13p\xa5\xa7A;\x99X\x18\xdf\xfd\x00oB\x93\xa1""\xa6""",2024-09-30 12:57:03,2024-09-30 12:57:04.785193,2024-09-30 12:57:04.785193,null,"""follow""",null,11877929974
859657,860200,"b""l\xe9\x81\xd7t\xf3\xd5\xc6\x17!4%\xfc|\xfd!<yf\xce""",2024-10-02 08:38:33,2024-10-02 08:38:36.010624,2024-10-02 08:38:36.010624,null,"""follow""",null,11878359754


In [36]:
# should filter by the links where the deleted at is null

In [ ]:
# add casts 

In [3]:
casts_lazy_df = pl.scan_parquet(f"{DATA_PATH}/farcaster-casts-0-1730134800.parquet")

In [24]:
casts_count_lazy = (
    casts_lazy_df
    .filter(pl.col("deleted_at").is_null())  # Filter out deleted casts
    .group_by("fid")
    .agg([
        pl.count("id").alias("total_casts_count"),
        (pl.col("parent_hash").is_not_null() & (pl.col("parent_hash").is_not_null())).sum().alias("reply_casts_count"),
        (pl.col("parent_hash").is_null()).sum().alias("casts_count"),
    ])
)

In [25]:
casts_count_df = casts_count_lazy.collect()

In [26]:
casts_count_df

fid,total_casts_count,reply_casts_count,casts_count
i64,u32,u32,u32
254061,163,120,43
617915,51,32,19
510194,3660,3284,376
247892,4,1,3
312523,3,0,3
…,…,…,…
683975,18,18,0
289541,242,153,89
379327,4272,3859,413


In [27]:
df = pl.read_parquet(f"{DATA_PATH}/aggregated_links_statistics.parquet")

In [33]:
# df = df.drop(["casts_count","casts_count_right"])
# df

In [31]:
casts_count_df.sample(10)

fid,total_casts_count,reply_casts_count,casts_count
i64,u32,u32,u32
772824,508,390,118
780412,31,15,16
829404,1,0,1
421163,24807,22349,2458
519541,12,10,2
854364,28,17,11
806023,33,3,30
262031,44,22,22
253708,12,4,8


In [34]:
result_df = df.join(casts_count_df, on="fid", how="left")
result_df = result_df.fill_null(0)

In [35]:
result_df.write_parquet(f"{DATA_PATH}/aggregated_links_statistics.parquet")

In [21]:
# casts_by_fid = casts_lazy_df.filter((pl.col('fid')==400242) & (pl.col('deleted_at').is_null()))
# result = casts_by_fid.collect()

In [20]:
# result.filter(~pl.col("root_parent_url").str.contains("https://warpcast.com/~/channel"))

created_at,updated_at,deleted_at,timestamp,fid,hash,parent_hash,parent_fid,parent_url,text,embeds,mentions,mentions_positions,root_parent_hash,root_parent_url,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,binary,i64,str,str,str,str,str,binary,str,i64
2024-03-24 07:08:16.583831,2024-03-24 07:08:16.583831,null,2024-03-24 07:08:17,400242,"b""|\xb7\x8d\xecx\x982\xd4`\x913\xdf\xbd\x1a\xb1(}\x85^(""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe with $DEGEN or ETH""","""[{""url"": ""https://toka.lol/collect/base:0x6b1ce2699455a096fb617a0d1850467c827979d5/2""}, {""castId"": {""fid"": 249927, ""hash"": {""data"": [100, 232, 88, 29, 234, 12, 108, 32, 172, 164, 63, 14, 191, 130, 10, 24, 243, 47, 217, 182], ""type"": ""Buffer""}}}]""","""[249927]""","""[17]""","b""|\xb7\x8d\xecx\x982\xd4`\x913\xdf\xbd\x1a\xb1(}\x85^(""","""https://warpcast.com/~/channel/toka""",963501345
2024-03-24 07:08:24.584160,2024-03-24 07:08:24.584160,null,2024-03-24 07:08:25,400242,"b""\xa2\xa1\xeaP\xda\xff\xec\x80\xfd\x9f%\x0b\xc3\x9dG\xaeN\x8eN\xef""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe with $DEGEN or ETH""","""[{""url"": ""https://toka.lol/collect/base:0x82e30a63bccde3724877878a30c4977b52348198/1""}, {""castId"": {""fid"": 281295, ""hash"": {""data"": [249, 12, 49, 117, 91, 171, 69, 68, 160, 129, 113, 43, 196, 132, 220, 81, 130, 151, 32, 91], ""type"": ""Buffer""}}}]""","""[293263]""","""[17]""","b""\xa2\xa1\xeaP\xda\xff\xec\x80\xfd\x9f%\x0b\xc3\x9dG\xaeN\x8eN\xef""","""https://warpcast.com/~/channel/toka""",963505914
2024-03-24 07:09:02.292216,2024-03-24 07:09:02.292216,null,2024-03-24 07:09:02,400242,"b""\xc8\xf7;\xa4\xa8Q\xf2\xb2\x9d\xc6\x87,-\xbd{9\xaaI\xa8\x0a""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe with $DEGEN or ETH""","""[{""url"": ""https://toka.lol/collect/base:0x3972af0ebfa3ffbfa0913243a5540c18308a9ca9/1""}, {""castId"": {""fid"": 292426, ""hash"": {""data"": [220, 53, 3, 253, 197, 220, 63, 206, 157, 135, 236, 152, 212, 18, 140, 241, 107, 26, 178, 128], ""type"": ""Buffer""}}}]""","""[5860]""","""[17]""","b""\xc8\xf7;\xa4\xa8Q\xf2\xb2\x9d\xc6\x87,-\xbd{9\xaaI\xa8\x0a""","""https://warpcast.com/~/channel/toka""",963515943
2024-03-24 07:19:48.723145,2024-03-24 07:19:48.723145,null,2024-03-24 07:19:49,400242,"b""Y'\x935\xa4{\x85y\xb9Z\xe69\x1d!Y\x84\x95!\x1c\xdc""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe with $DEGEN or ETH""","""[{""url"": ""https://toka.lol/collect/base:0x3f07b910f892fc1d19547d63ced91719dc595e7a/1""}, {""castId"": {""fid"": 328795, ""hash"": {""data"": [58, 203, 248, 180, 206, 40, 55, 255, 56, 98, 206, 175, 252, 1, 216, 62, 33, 80, 196, 127], ""type"": ""Buffer""}}}]""","""[328795]""","""[17]""","b""Y'\x935\xa4{\x85y\xb9Z\xe69\x1d!Y\x84\x95!\x1c\xdc""","""https://warpcast.com/~/channel/toka""",963751753
2024-03-24 07:38:57.714247,2024-03-24 07:38:57.714247,null,2024-03-24 07:38:58,400242,"b"":\xa1\xbc\x08\xb5k\xabl+=\xe6{<1\xb0\x0aZ\xe1r\x0f""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe with $DEGEN or ETH""","""[{""url"": ""https://toka.lol/collect/base:0x2d8f5a5530e6e8c0b53f2c39947cf0b3229cedeb/1""}, {""castId"": {""fid"": 236715, ""hash"": {""data"": [96, 190, 80, 213, 203, 203, 58, 43, 92, 175, 228, 192, 28, 9, 66, 60, 164, 45, 182, 249], ""type"": ""Buffer""}}}]""","""[236670]""","""[17]""","b"":\xa1\xbc\x08\xb5k\xabl+=\xe6{<1\xb0\x0aZ\xe1r\x0f""","""https://warpcast.com/~/channel/toka""",964171868
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-10-03 16:40:05.725472,2024-10-03 16:40:05.725472,null,2024-10-03 16:40:06,400242,"b""\xe5\x8b\x90\x84)\xc8\x98\x1b\xd1\x80bd\xceY\xb6\xb0\xf3\x05}\xda""",null,null,"""https://warpcast.com/~/channel/toka""","""Mint this NFT by completely onframe""","""[{""url"": ""https://toka.lol/collect/base:0x53dc7139cb12aaff78e9cea97fca9fcbdc08d8d

In [11]:
# b"\xed=\x85\x89\xeb\x89\xe8*\x1d\xa6P\x83\xe6[\xc9\x1af\x87IA"	.hex()

'ed3d8589eb89e82a1da65083e65bc91a66874941'

In [23]:
df_filtered = links_df.filter(
            (pl.col("deleted_at").is_null()) & 
            (pl.col("fid") != pl.col("target_fid")) & 
            (pl.col("type") == "follow")
        ).select([
            "fid", "target_fid"
        ])

In [24]:
mutual_links = df_filtered.join(
    df_filtered,
    left_on=["fid", "target_fid"],
    right_on=["target_fid", "fid"],
    suffix="_reverse"
)

KeyboardInterrupt: 

In [25]:
df_filtered = df_filtered.with_columns([
        pl.col("fid").cast(pl.Categorical),
        pl.col("target_fid").cast(pl.Categorical)
    ])

ComputeError: cannot cast numeric types to 'Categorical'

In [ ]:
mutual_links = df_filtered.join(
        df_filtered,
        left_on=["fid", "target_fid"],
        right_on=["target_fid", "fid"],
        suffix="_reverse",
    )